### Beginning

In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

: 

In [ ]:
from former import Tab_Former, log_rmse, run_model, Encoder

: 

In [ ]:
clean = pd.read_csv("Flight_prediction\Clean_Dataset.csv")
economy = pd.read_csv("Flight_prediction\economy.csv")
businesss = pd.read_csv("Flight_prediction/business.csv")

: 

In [ ]:
clean.head()

: 

In [ ]:
economy.head()

: 

In [ ]:
businesss.tail()

: 

In [ ]:
businesss.shape

: 

In [ ]:
188259 - 93487

: 

In [ ]:
len(clean['flight'].unique())

: 

### Preprocessing

In [ ]:
df = pd.concat([economy,businesss],axis=0)

: 

In [ ]:
df.head()

: 

In [ ]:
df['date'] = pd.to_datetime(df['date'],format = "%d-%m-%Y")

: 

In [ ]:
df.head()

: 

In [ ]:
# df.set_index('date',inplace=True)
# df.head()

: 

In [ ]:
def stop_data(data):
    for i in data:
        if 'non-stop' in i:
            return 'zero'
        elif '1' in i:
            return 'one'
        elif '2' in i:
            return 'two'

    return data

: 

In [ ]:
df['stop'] = df['stop'].apply(stop_data)

: 

In [ ]:
df.head()

: 

In [ ]:
df['arr_time'] = pd.to_datetime(df['arr_time'],format = "%H:%M")

: 

In [ ]:
df['dep_time'] = pd.to_datetime(df['dep_time'],format = "%H:%M")
# df['dep_time'].dt.time

: 

In [ ]:
df.head()

: 

In [ ]:
df['duration'] = df['arr_time']  - df['dep_time']

: 

In [ ]:
df['duration'] = df['duration'] / np.timedelta64(1,'h')

: 

In [ ]:
data = df['arr_time'].dt.time

: 

In [ ]:
data

: 

In [ ]:
bins = [pd.to_datetime(['00:00:00'],format='%H:%M:%S'),pd.to_datetime(['05:00:00'],format='%H:%M:%S'),
        pd.to_datetime(['12:00:00'],format='%H:%M:%S'), pd.to_datetime(['17:00:00'],format='%H:%M:%S'),
        pd.to_datetime(['21:00:00'],format='%H:%M:%S'), pd.to_datetime(['23:59:59'],format='%H:%M:%S')]
labels = ["Early Morning", "Morning", "Afternoon", 'Evening', 'Late Night']

# for time in df['dep_time']:
#     print(time.time())
#     pd.cut(x = time.time(), bins = bins, labels=labels)



: 

In [ ]:
df['dep_time']

: 

In [ ]:
# pd.cut(x = df['dep_time'], bins = bins, labels=labels)

: 

In [ ]:
df['arr_time']

: 

In [ ]:
df

: 

In [ ]:
timestamp = pd.Timedelta('02:00:00')
time = pd.to_datetime('20:00:00',format='%H:%M:%S')


if time >= pd.to_datetime('00:00:00',format='%H:%M:%S') and time < pd.to_datetime('05:00:00',format='%H:%M:%S'):
    print( "Early Morning")
elif time >= pd.to_datetime('05:00:00',format='%H:%M:%S') and time < pd.to_datetime('12:00:00',format='%H:%M:%S'):
    print( "Morning")
elif time >= pd.to_datetime('12:00:00',format='%H:%M:%S') and time < pd.to_datetime('19:00:00',format='%H:%M:%S'):
    print( "Afternoon")
elif time >= pd.to_datetime('19:00:00',format='%H:%M:%S') and time < pd.to_datetime('21:00:00',format='%H:%M:%S'):
    print( "Night")
    # print(time.time)
else:
    print( "Late Night")

: 

In [ ]:
def categorize_time(time_series):
    categories = []
    for time in time_series.dt.time:
        time = pd.to_datetime(time,format="%H:%M:%S")
        if time >= pd.to_datetime('00:00:00',format="%H:%M:%S") and time < pd.to_datetime('05:00:00',format="%H:%M:%S"):
            categories.append("Early Morning")
        elif time >= pd.to_datetime('05:00:00',format="%H:%M:%S") and time < pd.to_datetime('12:00:00',format="%H:%M:%S"):
            categories.append("Morning")        
        elif time >= pd.to_datetime('12:00:00',format="%H:%M:%S") and time < pd.to_datetime('19:00:00',format="%H:%M:%S"):
            categories.append("Afternoon")        
        elif time >= pd.to_datetime('19:00:00',format="%H:%M:%S") and time < pd.to_datetime('21:00:00',format="%H:%M:%S"):
            categories.append("Night")        
        else:
            categories.append("Late Night")
        

    return categories
        


: 

In [ ]:
dep = categorize_time(df['dep_time'])

: 

In [ ]:
arr = categorize_time(df['arr_time'])

: 

In [ ]:
df['Departure_time'] = pd.DataFrame(dep)
# df['Departure_time'] = de.values
df['Arrival_time'] = pd.DataFrame(arr)

: 

In [ ]:
df.tail()

: 

In [ ]:
df.dtypes

: 

In [ ]:
def convert_to_days(duration_str):
    hours, min = map(int,duration_str.split('h'))
    
    total_min = hours * 60 + min
    return total_min / (24 * 60)
    

: 

In [ ]:
df['duration_time'] = df['time_taken'].apply(convert_to_days)
df.head()

: 

In [ ]:
df[df['duration']>24]

: 

In [ ]:
df['duration'].max()

: 

In [ ]:
clean['duration'].max()

: 

In [ ]:
clean.tail()

: 

In [ ]:
clean.describe(include = 'all')

: 

In [ ]:
clean.info()

: 

In [ ]:
tab = Tab_Former(train_df = clean, test_df= None,target = "price",id = "Unnamed: 0")

: 

In [ ]:
combined_df, target = tab.preprocess_data()

: 

In [ ]:
print(combined_df)

: 

In [ ]:
combined_df = pd.concat([clean, None], axis=0)

: 

In [ ]:
combined_df

: 

In [ ]:
combined_df.select_dtypes(include='object').columns.tolist()

: 

In [ ]:
cat_col, num_col = tab.cat_num_split(combined_df)

: 

In [ ]:
print(cat_col)

: 

In [ ]:
feature_map = tab.feature_map()
mapp = len(feature_map)

: 

In [ ]:
print(feature_map)

: 

In [ ]:
from sklearn.model_selection import GridSearchCV

: 

In [ ]:
from former import Scaling

: 

In [ ]:
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.linear_model import LinearRegression
rfr = RandomForestRegressor(n_estimators=200,max_leaf_nodes=60)
IS = IsolationForest()
lr = LinearRegression()

: 

In [ ]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import catboost as cb
from sklearn.neural_network import MLPRegressor

: 

In [ ]:
models = {
    'RandomForest': RandomForestRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'XGBoost': xgb.XGBRegressor(),
    'LightGBM': lgb.LGBMRegressor(),
    'CatBoost': cb.CatBoostRegressor(silent=True),
    "LinearRegression" : LinearRegression(),
    "MLP" : MLPRegressor()
}


: 

In [ ]:
def results():
    result = []
    for name, model in models.items():
        mod = run_model(model,train,target)
        df = pd.DataFrame([run_model(model,train,target)])
        result.append(mod)

    df = pd.DataFrame(result)
    df.set_index('Model Name',inplace=True)
    
    plt.figure(figsize = (12,8))
    df['rmse'].plot()
    plt.title("RMSE per each model")
    plt.ylabel("RMSE")
    plt.show()

    return df

: 

### Expand blocks Layer in Encoder `1`, `2`, `4`, `8`, `12`, `50`, `100`.

#### Layer 1

In [ ]:
d_model = len(cat_col)
print(d_model)

: 

In [ ]:
num_layer = 1

: 

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=num_layer,d_ff=128,dropout=0.8)
enc_val = tab.encode()

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
df1 = results()

: 

In [ ]:
df1.add_prefix((f"{num_layer}"), axis=1)

: 

#### Layer 2

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=2,d_ff=128,dropout=0.5)
enc_val = tab.encode()

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 4

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=4,d_ff=128,dropout=0.5)
enc_val = tab.encode()

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 8

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=8,d_ff=128,dropout=0.5)
enc_val = tab.encode()

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 12

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=12,d_ff=128,dropout=0.5)
enc_val = tab.encode()

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 50

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=50,d_ff=128,dropout=0.5)
enc_val = tab.encode()

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 100

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=100,d_ff=128,dropout=0.5)
enc_val = tab.encode()

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

### Scale the numerical values `min_max_scaling`, `z_score_scaling`, `robust_scaling`.

#### Encoding before scaling

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=8,d_ff=128,dropout=0.5)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

#### min_max_scaling

In [ ]:
scale = Scaling.min_max_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
target

: 

In [ ]:
train

: 

#### z_score_scaling 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Robust Scaling

In [ ]:
from sklearn.preprocessing import RobustScaler

: 

In [ ]:
rs = RobustScaler()

: 

In [ ]:
scale = Scaling.robust_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

### Dropout in Encoder `0.2`, `0.4`, `0.8`.

#### Encoding with 0.2 dropout

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=8,d_ff=128,dropout=0.2)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.min_max_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Encoding with 0.4 dropout

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=8,d_ff=128,dropout=0.4)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.min_max_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Encoding with 0.8 dropout

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=8,d_ff=128,dropout=0.8)
enc_val = tab.encode(encoder = encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.min_max_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Dropout 0.5

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=27,num_layers=8,d_ff=128,dropout=0.5)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

### Another

#### min_max_scaling

In [ ]:
scale = Scaling.min_max_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### z_score_scaling 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Robust Scaling

In [ ]:
scale = Scaling.robust_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

### Expand Hidden Layer in Encoder `50`, `128`, `256`, `512`, `768`, `1024`, `2048`.

#### Layer 50

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=50,dropout=0.5)
enc_val = tab.encode(encoder = encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 128 

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=128,dropout=0.5)
enc_val = tab.encode(encoder = encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 256

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=256,dropout=0.5)
enc_val = tab.encode(encoder = encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 512

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=512,dropout=0.5)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 768

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=768,dropout=0.5)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 1024

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=1024,dropout=0.5)
enc_val = tab.encode(encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
target

: 

In [ ]:
train

: 

In [ ]:
results()

: 

#### Layer 2048

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=2048,dropout=0.5)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=len(cat_col), truncation=True, padding='max_length')
    outputs = model(**inputs)
    # Taking the mean of the embeddings from the last hidden state
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Encode categorical features
# cat_features = ['categorical_feature1', 'categorical_feature2']

: 

In [ ]:
encoded_feature = combined_df[cat_col]
encoded_feature.head()    

: 

In [ ]:
import pandas as pd

# Example DataFrame
data = {
    'categorical_feature1': ['cat1', 'cat2', 'cat1'],
    'categorical_feature2': ['cat3', 'cat4', 'cat3'],
    'numerical_feature1': [1.0, 2.0, 3.0],
    'numerical_feature2': [4.0, 5.0, 6.0],
    'target': [0, 1, 0]
}

df = pd.DataFrame(data)
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings
def get_bert_embeddings(text):
    text = ' '.join([str(text) for feature in text])
    inputs = tokenizer(text, return_tensors='pt', max_length=3, truncation=True, padding='max_length')
    outputs = model(**inputs)
    # Taking the mean of the embeddings from the last hidden state
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()


: 

In [ ]:
train_df

: 

In [ ]:
# cat_features = ['categorical_feature1', 'categorical_feature2']
encoded_features = []

for feature in cat_col:
    encoded_feature = get_bert_embeddings(train_df[feature])
    encoded_features.append(encoded_feature)

# Concatenate encoded categorical features
encoded_cat_df = pd.DataFrame(encoded_features[0].tolist()).add_prefix(f"{cat_col[0]}_emb_")
for i, feature in enumerate(cat_col[1:]):
    encoded_cat_df = pd.concat([encoded_cat_df, pd.DataFrame(encoded_features[i+1].tolist()).add_prefix(f"{feature}_emb_")], axis=1)
# Numerical features
# numerical_features = ['numerical_feature1', 'numerical_feature2']
numerical_df = train_df[num_col]

# Concatenate encoded categorical features with numerical features
final_df = pd.concat([encoded_cat_df, numerical_df], axis=1)

: 

In [ ]:
tab = Tab_Former(train_df=train_df,test_df=None, target = "SalePrice")
tab.preprocess_data()

: 

In [ ]:
final_df.head()

: 

In [ ]:
missing=final_df.isnull().sum().sort_values(ascending=False)
missing=missing.drop(missing[missing==0].index)
df = pd.DataFrame(missing)
label = train_df['SalePrice']
# test_id = test_df['Id']
# train_df.drop("Id",inplace=True,axis=1)
final_df.drop(missing.keys(),inplace=True,axis=1)

: 

In [ ]:
len(final_df.isna().sum())

: 

: 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Target variable
# y = df['target']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(final_df, label, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
clf = RandomForestRegressor()
clf.fit(X_train, y_train)

# Evaluate the model
accuracy = clf.score(X_test, y_test)
print(f"Model Accuracy: {accuracy}")


: 

In [ ]:
for i in range(combined_df.shape[0]):
    get_bert_embeddings(combined_df[cat_col].iloc[i])

: 

In [ ]:
encoded_features = []

for feature in cat_col:
    encoded_feature = get_bert_embeddings(combined_df[feature].)
    encoded_features.append(encoded_feature)

# Concatenate encoded categorical features
encoded_cat_df = pd.DataFrame(encoded_features[0].tolist()).add_prefix(f"{cat_col[0]}_emb_")
for i, feature in enumerate(cat_col[1:]):
    encoded_cat_df = pd.concat([encoded_cat_df, pd.DataFrame(encoded_features[i+1].tolist()).add_prefix(f"{feature}_emb_")], axis=1)


: 

### Others

In [ ]:
# Define custom dataset class
class AnyDataset(Dataset):
    def __init__(self, cat_features, labels, tokenizer, max_length):
    #     self.num_features = num_features
        self.cat_features = cat_features
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.cat_features)

    def __getitem__(self, idx):
        text = ' '.join([str(feature) for feature in self.cat_features[idx]])
        label = self.labels[idx]

        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }



: 

In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer

: 

In [ ]:
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


: 

In [ ]:
def hyperparameter(train,MAX_LENGTH=128,BATCH_SIZE=32,num_epochs=5,label=None):
    """Initializing the Hyperparameters and Running the model

      parameters = dataset,
                    MAX_LENGTH,
                    BATCH_SIZE,
                    num_epochs,
                    label
                    """

    X_train, X_val, y_train, y_val = train_test_split(train,target,test_size=0.25,random_state=24)

    # Create custom datasets and data loaders
    train_dataset = AnyDataset(X_train, y_train, tokenizer, MAX_LENGTH)
    val_dataset = AnyDataset(X_val, y_val, tokenizer, MAX_LENGTH)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Fine-tune BERT model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.train()

    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    # print(next(iter(train_dataset)))

    torch.manual_seed(42)
    for epoch in range(num_epochs):
      for batch in train_loader:
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['label'].to(device)

          outputs = model(input_ids, attention_mask=attention_mask)
          logits = outputs.last_hidden_state[:, 0, :]  # Get the CLS token representation
          # predictions = torch.argmax(logits, dim=1)
          prediction = outputs.Softmax(logits,dim=27)

          loss = torch.nn.L1Loss()(logits, labels)

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      print(f"Loss: {loss}")

      # Evaluate model on validation set
      model.eval()
      val_predictions = []
      val_labels = []

      for batch in val_loader:
          with torch.no_grad():
              input_ids = batch['input_ids'].to(device)
              attention_mask = batch['attention_mask'].to(device)
              labels = batch['label'].to(device)

              outputs = model(input_ids, attention_mask=attention_mask)
              logits = outputs.last_hidden_state[:, 0, :]
              # predictions = torch.argmax(logits, dim=1)
              predictions = outputs.Softmax(logits,dim=27)

              val_predictions.extend(predictions.cpu().numpy())
              val_labels.extend(labels.cpu().numpy())

      # Calculate accuracy
      # accuracy = log(val_labels, val_predictions)/
      # print(f'Validation Accuracy: {accuracy}')

    return {"model_name": model.__class__.__name__,
            "model_loss": loss.item(),
            # "model_acc": accuracy,
            'token_length':MAX_LENGTH,
            'batch_size':BATCH_SIZE}

: 

In [ ]:
hyperparameter(train_df,len(cat_col),label="SalePrice")

: 

: 